In [ ]:
import panel as pn
import pandas as pd
import hvplot.pandas

Defer loading of the data and populating the widgets until the page is rendered:

In [ ]:
stocks_url = 'https://raw.githubusercontent.com/vega/datalib/master/test/data/stocks.csv'

select_ticker = pn.widgets.Select(name='Stock Ticker')

def load_data():
    if 'stocks' not in pn.state.cache: 
        pn.state.cache['stocks'] = df = pd.read_csv(stocks_url, parse_dates=['date']).set_index('symbol')
    else:
        df = pn.state.cache['stocks']
    symbols = list(df.index.unique())
    select_ticker.options = symbols
    select_ticker.value = symbols[0]
    
pn.state.onload(load_data)

If `'stocks'` is not yet in cache we show a spinner widget, otherwise let us plot the data:

In [ ]:
@pn.depends(select_ticker)
def plot_ticker(ticker):
    if 'stocks' not in pn.state.cache or not ticker:
        return pn.indicators.LoadingSpinner(value=True)
    return pn.state.cache['stocks'].loc[ticker].hvplot.line('date', 'price')

pn.Row(select_ticker, plot_ticker).servable()